## 1: Setup and Dependencies

This notebook sets up your environment for deploying agents to production using AgentCore Runtime. You'll install dependencies, configure AWS credentials, and prepare Memory and Knowledge Base resources.

**Prerequisites:** Python 3.12+, AWS account with Bedrock permissions

### Step 1: Install Dependencies and Import Required Libraries

In [ ]:
# Install required packages
%pip install -U -r requirements.txt -q

In [ ]:
# Import required libraries and verify AWS configuration
import json
import boto3
from strands import Agent
from strands.models import BedrockModel

session = boto3.Session()
sts = session.client('sts')
identity = sts.get_caller_identity()
account_id = identity['Account']
region = session.region_name or 'us-west-2'

print(f"✅ Account ID: {account_id}")
print(f"✅ Region: {region}")

### Step 2: Prepare Memory and Knowledge Base

This creates AgentCore Memory with three strategies: User Preferences, Semantic memory for conversation facts, and Summary memory for conversation context.

**Note:** This may take a few minutes. While waiting, try asking Kiro (chat on right) about AgentCore Memory concepts.

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

MEMORY_NAME="ReturnRefundAssisantMemory"

memory_manager = MemoryManager(region_name=region)

# Create or get memory with strategies
memory = memory_manager.get_or_create_memory(
    name=MEMORY_NAME,
    description="Memory for returns and refunds assistant",
    strategies=[
                {
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "CustomerPreferences",
                        "description": "Captures customer preferences and behavior",
                        "namespaces": ["returns/customer/{actorId}/preferences"],
                    }
                },
                {
                    StrategyType.SEMANTIC.value: {
                        "name": "CustomerSupportSemantic",
                        "description": "Stores facts from conversations",
                        "namespaces": ["returns/customer/{actorId}/semantic"],
                    }
                },
                {
                    StrategyType.SUMMARY.value: {
                        "name": "ConversationSummary",
                        "description": "Maintains conversation context and summaries",
                        "namespaces": ["returns/customer/{actorId}/{sessionId}/summary"],
                    }
                },
            ]
)

memory_id = memory["id"]

# Save memory configuration to JSON file
memory_config = {
    "memory_id": memory_id,
    "memory_name": MEMORY_NAME
}
with open('memory_config.json', 'w') as f:
    json.dump(memory_config, f, indent=2)

print(f"✅ Memory configuration saved to memory_config.json")

### Step 3: Retrieve Knowledge Base ID

Get the Knowledge Base ID from the CloudFormation stack output. This Knowledge Base contains the return and refund policies.

In [ ]:
# Retrieve the knowledge base ID from CloudFormation stack outputs
cfn_client = boto3.client('cloudformation', region_name=region)

try:
    response = cfn_client.describe_stacks(StackName='knowledgebase')
    outputs = response['Stacks'][0]['Outputs']
    
    kb_id = None
    for output in outputs:
        if output['OutputKey'] == 'KnowledgeBaseId':
            kb_id = output['OutputValue']
            break
    
    if kb_id:
        print(f"✅ Knowledge Base ID: {kb_id}")
    else:
        raise ValueError("KnowledgeBaseId output not found in stack")
        
except Exception as e:
    print(f"⚠️ Error retrieving from CloudFormation: {e}")
    kb_id = "<PLACE-YOUR-KB-ID>"

# Save knowledge base configuration to JSON file
kb_config = {
    "kb_id": kb_id
}
with open('kb_config.json', 'w') as f:
    json.dump(kb_config, f, indent=2)

print(f"✅ Knowledge Base configuration saved to kb_config.json")

### Summary

You've installed dependencies, configured AWS credentials, and created Memory and Knowledge Base resources.

### Next Steps

- **2: AgentCore Identity** - Configure secure authentication